# Visualización de un Autómata de Segundo y Primer orden

In [ ]:
# Original Title: SecondOrder1dCA.ipynb
# Original Author: Jorge Luis Rosas Trigueros
# Visualizes the evolution of a 1d CA with 1st and 2nd order rules
# Last modification: 10 nov 21 20:23

import cv2
import numpy as np
from IPython import display as display
import ipywidgets as ipw
import PIL
from io import BytesIO
from random import random

In [ ]:
class Automaton:
    def __init__(self, grid_width, neighborhood_width, rule, random_init: False, is_second_order: False):
        self.grid_width = grid_width
        self.neighborhood_width = neighborhood_width
        self.is_second_order = is_second_order

        self.last_iteration = grid_width * [0]
        self.current_iteration = [0 if random()>0.5 else 1 for _ in range(grid_width)] if random_init else grid_width * [0]
        self.new_iteration = grid_width * [0]

        self.rule = (22*[0]).extend(rule)
        self.rule = self.rule[::-1]   # Se pone al revés

        self.x0 = 0
        self.y0 = 0
        self.maxX = 500
        self.maxY = 500
        self.color = (255,255,255)
        self.margin = 1
        self.stride = (self.maxX - 2*self.margin) / self.grid_width
        self.wIm = ipw.Image()
        display.display(self.wIm)
        self.img = np.zeros((500, 500, 3), dtype="uint8")
        self.graph_cells(self.img, self.current_iteration)
        self.y0 += self.stride


    def apply_rule(self, neighborhood):
        total_sum = 0

        for index in range(len(neighborhood)):
            cell = neighborhood[-(index + 1)]
            total_sum += (2 ** index) * cell

        return self.rule[total_sum]


    def iterate(self):
        while self.y0 <= self.maxY:
            if self.is_second_order:
                self.second_order_evaluation()
            else:
                self.second_order_evaluation()

            self.graph_cells(self.img,self.last_iteration)
            self.y0 += self.stride

        pilIm = PIL.Image.fromarray(self.img, mode="RGB")
        with BytesIO() as fOut:
            pilIm.save(fOut, format="png")
            byPng = fOut.getvalue()

        self.wIm.value = byPng


    def first_order_evaluation(self):
        for i in range(self.neighborhood_width, self.grid_width-self.neighborhood_width):
            self.last_iteration[i] = self.apply_rule(self.current_iteration[i-self.neighborhood_width : i+self.neighborhood_width])
        self.current_iteration[:] = self.last_iteration[:]


    def second_order_evaluation(self):
        for i in range(self.neighborhood_width, self.grid_width-self.neighborhood_width):
            self.last_iteration[i] = self.apply_rule(self.current_iteration[i-self.neighborhood_width : i+self.neighborhood_width])
            self.last_iteration[i] = 0 if self.last_iteration[i] == self.new_iteration[i] else 1

        self.new_iteration[:] = self.current_iteration[:]
        self.current_iteration[:] = self.last_iteration[:]


    def graph_cells(self, img, cells):
        for i in range(self.grid_width):
            if cells[i]:
                start = (int(self.x0 + self.stride*i + self.margin), int(self.y0 + self.margin))
                end = (int(self.x0 + self.stride*(i+1) - self.margin), int(self.y0 + self.stride-self.margin))
                cv2.rectangle(img, start, end, self.color, -1)

In [1]:
GRID_WIDTH = 100
NEIGHBORHOOD_WIDTH = 2
RULE = [1,1,0,1,0,0,1,0,0,0]          # Está escrita al revés

automaton = Automaton(GRID_WIDTH, NEIGHBORHOOD_WIDTH, RULE, random_init=True, is_second_order=False)
automaton.iterate()

[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Image(value=b'')